# The Cantor-Thirds Counterexample

 > **Warning:** The following code creates several three dimensional interactive plots. To ensure the notebook remains responsive, it is highly recommended that each plot is cleared before the next block is executed.

### Installations

The below will install the necessary packages to run the notebook.

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install kaleido
!pip install plotly
!pip install scipy

The following block imports the relevant modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.integrate import cumulative_trapezoid
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

The below helper functions will be useful later.

In [ ]:
from src.integral import compute_integral
from src.plot import plot_surface_3d, plot_slice_of_f_star, plot_z_slice_of_f
from src.H_construct import spike, compute_h, compute_H_tilde, compute_H_star, compute_H_star_global
from src.T_construct import compute_T_tilde, compute_T_star
from src.f_construct import compute_f_star
from src.extend import extend_F
from src.coord_transform import spherical_to_cartesian

### Introduction

In this notebook, we shall construct a continuously differentiable function $f:\mathbb{R}^3\rightarrow \mathbb{R}$ with a single isolated critical point at the origin, such that for every $\epsilon > 0$, the level set $f^{-1}(0)$ intersects the boundary of the $\epsilon$-ball, $\partial B_\epsilon$, tangentially. The existence of such a function motivates the inclusion of Lemma 3 in the main text, which rules such functions from consideration in the proof of Theorem 3.

To achieve this goal, we construct the function in two steps, with intermediate and final expressions given in spherical coordinates $(r, \theta, \psi)$, where $r \in [0,\infty)$, $\theta \in [0,\pi]$, and $\psi \in [0,2\pi)$:

- **Step 1:** Define a $C^1$ function $H : [0,\pi] \times [0, 2\pi) \rightarrow [0,\infty)$ such that the surface given by $r = H(\theta, \psi)$ is tangent to every ball of radius less than or equal to 1.

- **Step 2:** Construct a $C^1$ function $f : [0,\infty) \times [0,\pi] \times [0,2\pi) \rightarrow \mathbb{R}$ with zero level set given by
  
  $$f^{-1}(0) = \{ (r, \theta, \psi) \mid r = H(\theta, \psi) \},$$
  zero gradient at the origin, and non-vanishing gradient everywhere else within a neighbourhood of the origin.

We treat $[0,\infty) \times [0,\pi] \times [0,2\pi)$ as spherical coordinates on $\mathbb{R}^3$, with the usual identification of $\psi = 0$ and $\psi = 2\pi$, and the azimuthal degeneracy at the poles $\theta = 0$ and $\theta = \pi$. 


### Step 1: Constructing $H$

To construct $H$, we shall first define an auxiliary function $H^\ast : \mathbb{R}^2 \to \mathbb{R}$, which we will later map onto the sphere. $H^\ast$ is constructed so that for every $z \in [0,1]$, there exists a point $(x, y) \in [0,1]^2$ with $H^\ast(x, y) = z$ and $\nabla H^\ast(x, y) = 0$. This construction follows the approach of [Grinberg (2018)](https://www.tandfonline.com/doi/abs/10.1080/00029890.1985.11971725), but we provide explicit equations throughout.


To be specific, we construct $H$ in several steps:

1. **One-Dimensional slice:** We begin by defining a function $\tilde{H} : [0,1] \to [0,1]$ that has a critical point at every level $c \in C$, where $C$ denotes the middle-thirds Cantor set.

2. **Symmetrization:** We extend this function to $[-1,1]$ by requiring it to be even.

3. **Product Function:** We next define $H^\ast:[-1,1]^2\rightarrow [0,1]$ as follows:
   $$
   H^\ast(x, y) = \frac{1}{2}(\tilde{H}(x) + \tilde{H}(y)).
   $$
   The graph of this function is a two-dimensional surface with a critical point at every level $z\in[0,1]$.

5. **Global Extension:** Extending the definition of $H^\ast$ on $[-1,1]^2$ from above, we now define $H^\ast(x, y) = 1$ for all $(x, y) \in \mathbb{R}^2 \setminus [-2,2]^2$, and use a $C^1$ transition function to smoothly interpolate over the rectangular region $[-2,2]^2 \setminus [-1,1]^2$, resulting in a globally $C^1$ function.

6. **Mapping to Spherical:** We next define $H$ in terms of spherical coordinates $(\theta,\psi)$ as $H(\theta,\psi):=H^\ast(\frac{\pi}{2k}(\theta+k),\frac{\pi}{k}(\psi+k))$ for some suitably large $k$.

#### One-Dimensional Slice

To construct the 1 dimensional function with critical points at every level $c \in C$, we shall use a fractal construction for the derivative of $\tilde{H}$ which involves carefully placing and summing spikes of precomputed areas.

This spike construction shall then allow us to define a function whose integral has critical values at all levels in the Cantor set — a key fact which we shall later exploit to ensure that the level set of $f$ tangentially touches all small spheres around the origin.

We start by letting $n\in\mathbb{N}_{>0}$ and $k \in \{0, 1, \dots, 2^n - 1\}$, and write the binary expansion of $k$ as:

\begin{equation}\nonumber
k = \sum_{j=1}^n r_j \cdot 2^{n-j}, \quad \text{where } r_j \in \{0,1\}.
\end{equation}
We then define:
\begin{equation}\nonumber
a_{n,k} := \sum_{j=1}^n r_j \cdot \frac{3}{5} \cdot \left( \frac{2}{5} \right)^{j-1},\quad\text{and}\quad b_{n,k} := a_{n,k} + \left( \frac{2}{5} \right)^n.
\end{equation}


Next, we define the following 'spike' function for $x \in [0,1]$, $n \in \mathbb{N}_{>0}$, and $k \in \{0, \dots, 2^{n-1} - 1\}$:

$$
\text{Spike}_{k,n}(x) = 
\begin{cases}
    0 & \text{if } x \leq \frac{1}{2}(a_{n,k} + b_{n,k} - w_n), \\
    2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k}), \\
    h_n - 2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k}) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k}) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n), \\
    0 & \text{if } x \geq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n).
\end{cases}
$$

where the height and width are given by:

$$
h_n := 4 \left( \frac{5}{6} \right)^{n+1}, \qquad w_n := \frac{2}{h_n \cdot 3^{n+1}}.
$$

To give some intuition as to what this looks like, we provide the below example plot:

In [ ]:
# Set spike parameters
n = 0
k = 0

# Generate x-values for plotting
x_vals = np.linspace(0, 1, 1000)

# Evaluate the spike function
y_vals = spike(x_vals, n, k)

# Plot the spike function
plt.plot(x_vals, y_vals)
plt.title(f"Spike function for n = {n}, k = {k}")
plt.xlabel("x")
plt.ylabel(f"Spike_{{{k},{n}}}(x)")
plt.grid(True)
plt.show()

The 'spike' function creates a triangular signal, with carefully chosen width and height, which encloses an area of exactly $(\frac{2}{3})^{n+1}$.

The position of the spikes have been specified to ensure that all spikes have disjoint support. Noting this, we define a function $h$ as follows:

\begin{equation}\nonumber 
    h(x)= \sum_{n=0}^{\infty}\sum_{i=0}^{2^n-1} \text{Spike}_{k,n}(x).
\end{equation}

In [ ]:
# Generate x-values for plotting
x_vals = np.linspace(0, 1, 1000)

# Compute approximation of h(x) using spikes up to level N_max = 6
h_vals = compute_h(x_vals, N_max=6)

# Plot the function h(x)
plt.plot(x_vals, h_vals)
plt.title("Approximation of h(x) with N_max = 6")
plt.xlabel("x")
plt.ylabel("h(x)")
plt.grid(True)
plt.show()

The function $h$ is a fractal consisting of infinitely many non-overlapping spikes. Due to the choices of $h_n$, $w_n$, $a_{n,k}$, and $b_{n,k}$ used in the construction, the spikes become shorter and narrower as $n$ increases. It is a straightforward exercise to verify that $h$ is continuous.

The construction begins with a single spike of area $\frac{2}{3}$, followed by 2 spikes of area $\left(\frac{2}{3}\right)^2$, then 4 spikes of area $\left(\frac{2}{3}\right)^3$, and so on. Noting this, a simple calculation shows that the total area under the graph of $h$ over $[0,1]$ is exactly $1$.

Furthermore, as every $c \in C$ can be expressed as a base 3 expansion using only $0$s and $2$s, or equivalently as a linear combination of powers of $\frac{2}{3}$, it can be shown that, for all $c\in C$, there exists an $x \in [0,1]$ such that the cumulative area under $h$ from $0$ to $x$ is exactly $c$. This defines a bijection between $C$ and the set of zeros of $h$, where $h(x) = 0$ if and only if the cumulative area up to $x$ equals $c$.


Noting this, we now define $\tilde{H}:[0,1]\rightarrow [0,1]$ as follows:

\begin{equation}\nonumber
    \tilde{H}(x):=\int_{t=0}^x h(t)dt.
\end{equation}


In [ ]:
# Generate x-values for plotting
x_vals = np.linspace(0, 1, 1000)

# Compute the integral of h(x) to obtain \tilde{H}(x)
H_tilde_vals = compute_H_tilde(x_vals)

# Plot the function \tilde{H}(x)
plt.plot(x_vals, H_tilde_vals)
plt.title(r"Approximation of $\tilde{H}(x)$")
plt.xlabel("x")
plt.ylabel(r"$\tilde{H}(x)$")
plt.grid(True)
plt.show()

#### Symmetrization

We now extend the definition of $\tilde{H}$ to $[-1,1]$ by taking $\tilde{H}(x):=\tilde{H}(-x)$ for $x<0$. Note that, as $\tilde{H}'(0)=h(0)=0$, $\tilde{H}$ is continuously differentiable on $[-1,1]$.

In [ ]:
# Generate x-values over the symmetric domain [-1, 1]
x_vals = np.linspace(-1, 1, 1000)

# Compute the symmetrized \tilde{H}(x)
H_tilde_vals = compute_H_tilde(x_vals)

# Plot the function \tilde{H}(x)
plt.plot(x_vals, H_tilde_vals)
plt.title(r"Symmetric Extension of $\tilde{H}(x)$")
plt.xlabel("x")
plt.ylabel(r"$\tilde{H}(x)$")
plt.grid(True)
plt.show()

By construction, $\tilde{H}$ has critical points at every level of the Cantor set. 


#### Product Function

Next, we define a two-dimensional function $H^\ast:[-1,1]^2\rightarrow [-1,1]$ by $H^\ast(x,y):=\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$.

In [ ]:
# Compute values of H*(x, y) over the grid defined by x_vals
H_star_vals = compute_H_star(x_vals)

# Plot the 3D surface of H*(x, y)
plot_surface_3d(
    H_star_vals,
    x_vals,
    downsample=2,
    title="3D Surface with Critical Values in [0, 1]",
    z_title="H*(x, y) = (H̃(x) + H̃(y))/2"
)

By exploiting a well known function of the Cantor set; $C+C=[0,2]$, it can be seen that $H$ has a critical value at every level $z\in[0,1]$.

#### Global Extension

Next, we extend $H^\ast$ to the entirety of $\mathbb{R}^2$. This is achieved using the $C^1$ transition function $T(x)$ defined by:

$$
T(x):=\begin{cases}
0 & x \leq 0, \\
\frac{x^2}{x^2+(1-x)^2} & 0 \leq x \leq 1, \\
1 & 1 \leq x. \\
\end{cases}
$$

We begin by extending $\tilde{H}$ to $\mathbb{R}$ by letting $\tilde{H}(x)=1$ for $x \not\in[-1,1]^2$. Next, we extend the definition of $H^\ast$ as follows:

$$H^\ast(x,y):=T\bigg(d\big((x,y),[-1,1]^2\big)\bigg)+\frac{1}{2}\bigg(1-T\bigg(d\big((x,y),[-1,1]^2\big)\bigg)\bigg)(\tilde{H}(x)+\tilde{H}(y)),$$

where $d(p,A):=\inf_{a\in A}|p-a|$ represents the Euclidean distance from the point $p$ to the set $A$. 

The above notation is dense, but much easier to understand when visualised. When $d((x,y),[-1,1]^2)\geq 1$, we see that $H^\ast(x,y)=1$, and when $(x,y)\in [-1,1]^2$ we have that $d((x,y),[-1,1]^2)=0$ and thus $H^\ast(x,y)=\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$, as before. In the intermediate region, the function smoothly transitions from $\frac{1}{2}(\tilde{H}(x)+\tilde{H}(y))$ to $1$.

In [ ]:
# Define x and y values for the extended domain
x_vals = np.linspace(-6, 6, 1000)
y_vals = np.linspace(-6, 6, 1000)

# Compute the globally extended H*(x, y)
H_star_vals = compute_H_star_global(x_vals, y_vals)

# Plot the extended 3D surface of H*(x, y)
plot_surface_3d(
    H_star_vals,
    x_vals,
    downsample=2,
    title="3D Surface with Critical Values in [0, 1]",
    z_title="H*(x, y) = (H̃(x) + H̃(y)) / 2"
)

#### Mapping to Spherical

We next define $H$ in spherical coordinates $(\theta,\psi)$ by
$$
H(\theta,\psi) := H^*\left(\frac{\pi}{2k}(\theta + k),\, \frac{\pi}{k}(\psi + k)\right)
$$
for some suitably large integer $k$. Geometrically, this corresponds to embedding a plane containing the critical structure of $H^*$, onto the surface of a sphere of radius 1. If $k$ is large, intuitively, one can think of this as removing a small 'panel' from the sphere and replacing it with a rescaled and curved version of $H^*$ that conforms to the spherical geometry. In the following plots we take a small value of $k$, $k=4$, which means the 'panel' is large enough to cover approximately half of the sphere.

Before providing a 3D illustration, it is worth viewing the 2D analogue of the transformation, which is shown below.


In [ ]:
# Generate x-values for plotting
x_vals = np.linspace(-1, 1, 1000)

# Compute h(x) for Cartesian plot
h_vals = compute_H_tilde(x_vals)

# Convert x-values to theta for polar plot
theta_vals = (x_vals / 2 + 1) * np.pi

# Compute r-values for polar plot using the same function
r_vals = compute_H_tilde(x_vals)

# Set up the figure with two subplots
fig = plt.figure(figsize=(12, 5))

# Cartesian subplot
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(x_vals, h_vals)
ax1.set_title("Cartesian: y = H̃(x)")
ax1.set_xlabel("x")
ax1.set_ylabel("H̃(x)")
ax1.grid(True)

# Polar subplot
ax2 = fig.add_subplot(1, 2, 2, projection='polar')
ax2.plot(theta_vals, r_vals)
ax2.set_title("Polar: r = H̃(θ)")

# Adjust layout
plt.tight_layout()
plt.show()

In the above, we have treated the function $\tilde{H}:[-1,1]\rightarrow [0,1]$ as a function of polar coordinates rather than Cartesian coordinates. In doing so, we transform a function graph which is tangent to every plane of the form $y=c, c \in C$ to a function graph which is tangent to *every ball* of radius $c$ for $c\in C$.

The same trick is employed in three dimensions using $H^\ast$ to create the below plot. This plot illustrates the surface $r=H(\theta,\psi)$, which by construction is tangent to every sphere with radius in $[0,1]$.

In [ ]:
# Resolution for the grid
res = 1000

# Angular coordinates in spherical coordinates
# The small offset (0.2) avoids gaps from floating-point rounding
theta_vals = np.linspace(0, np.pi + 0.2, res)
psi_vals = np.linspace(0, 2 * np.pi + 0.2, res)

# Scaling factor for coordinate transformation
k = 4

# Convert spherical angles to x, y for computing H^*
x_vals = k * (2 * theta_vals / np.pi - 1)
y_vals = k * (psi_vals / np.pi - 1)

# Compute radial values using extended H^* on the transformed domain
R = compute_H_star_global(x_vals, y_vals)

# Create a meshgrid of spherical coordinates (theta, psi)
Theta, Psi = np.meshgrid(theta_vals, psi_vals, indexing='ij')

# Convert spherical coordinates to Cartesian for visualization
X = R * np.sin(Theta) * np.cos(Psi)
Y = R * np.sin(Theta) * np.sin(Psi)
Z = R * np.cos(Theta)

# Plot the surface in 3D
plot_surface_3d(
    Z, X, Y,
    downsample=2
)

Our goal is now to construct a function which has only a single isolated critical point at the origin and possesses the above spherical surface as its level set.

### Step 2: Constructing $f$

We now construct a function $f : [0,\infty) \times [0,\pi] \times [0,2\pi) \rightarrow \mathbb{R}$ whose zero level set corresponds to the surface defined by $H$, that is,
$$
f^{-1}(0) = \{ (r, \theta, \psi) \mid r = H(\theta, \psi) \},
$$
such that $f$ has vanishing gradient at the origin, and non-vanishing gradient in a neighborhood around the origin, excluding the origin itself.



To do so, we first define another function $T^*:\mathbb{R}^2\rightarrow [0,1]$. We construct $T^*$ in a similar fashion to $H^*$ using the previously defined $T:\mathbb{R}\rightarrow [0,1]$. First, we define $\tilde{T}:\mathbb{R}\rightarrow [0,1]$ as the even function $\tilde{T}(x):=T(|x|)$.

In [ ]:
# Generate x-values for plotting
x_values = np.linspace(-2, 2, 1000)

# Compute the even extension of T(x)
T_values = compute_T_tilde(x_values)

# Plot the function \tilde{T}(x)
plt.plot(x_values, T_values, label=r"$\tilde{T}(x)$")
plt.title(r"Plot of $\tilde{T}(x) = T(|x|)$")
plt.xlabel(r"$x$")
plt.ylabel(r"$\tilde{T}(x)$")
plt.grid(True)
plt.legend()
plt.show()

Next, we define $T^*:\mathbb{R}\rightarrow [0,1]$ to be equal to $\frac{1}{2}(\tilde{T}(x)+\tilde{T}(y))$ on $[-1,1]^2$, $1$ on $\mathbb{R}^2\setminus [-2,2]^2$. We define $T^*$ on $[-2,2]^2\setminus[-1,1]^2$ using the same $C^1$ extension procedure as used in the construction of $H^*$. The below properties are immediate consequences of the definition.

 - If $(x,y)\neq (0,0)$, then $T^*(x,y)>0$.
 - For $x\in(0,1)$, we have
   $$\frac{\partial T^*}{\partial x}(x,y)=\frac{1}{2}\tilde{T}'(x) >0,$$
   with the analogous statement holding for $y\in(0,1)$.

In [ ]:
# Generate x and y values over the square domain
x_vals = np.linspace(-1, 1, 400)
y_vals = np.linspace(-1, 1, 400)

# Compute the 2D extension T*(x, y) and transpose for plotting
T_vals = compute_T_star(x_vals, y_vals).T

# Plot the 3D surface of T*(x, y)
plot_surface_3d(
    T_vals,
    x_vals,
    y_vals,
    downsample=1,
    title="3D Surface of T*(x, y)",
    z_title="T*(x, y) = T̃(x) + T̃(y)"
)

Next, we define $f^\ast:\mathbb{R}^2 \times \mathbb{R}_{\geq 0} \rightarrow \mathbb{R}$ in cartesian coordinates as:

$$f^\ast(x,y,z):=-T^\ast(x,y)(z-H^\ast(x,y))z^2-\mathbb{1}[z> H^\ast(x,y)](z-H^\ast(x,y))^2,$$

where $\mathbb{1}[A]$ is the indicator function which evaluates to $1$ if the $A$ is true and $0$ otherwise.

The above is difficult to visualise. To aid understanding the below function creates 3D plots of different slices of $f^\ast$. For instance, to view the slice $x=c$ you can use the inputs `axis='x'` and `slice_value=c`.

In [ ]:
# Plot a slice of f* at y = 0.2
plot_slice_of_f_star(axis='y', slice_value=0.2)

Note that as $f^\ast$ is a composition of $C^1$ functions, it is $C^1$. We now verify three key properties of $f^\ast$, which ensure the zero level set behaves as required and that the gradient vanishes only at the origin.

 1. The preimage under $f^\ast$ of $0$ is
    $$(f^\ast)^{-1}(0)=\{(x,y,z):z=H^\ast(x,y)\}\cup \{(x,y,z):z=0\}.$$
 2. For all $x,y\in\mathbb{R}$, the gradient of $f^\ast$ at $(x,y,0)$ is
    $$\nabla f^\ast(x,y,0)=0.$$
 3. For all $x,y\in\mathbb{R}$ and $z\in(0,\frac{1}{3})$, the gradient of $f^\ast$ at $(x,y,z)$ is nonzero.

These properties shall also be crucial in the following sections, where we shall consider the function $f^\ast$ applied to spherical coordinates.

##### Proof of Property 1

Consider $(x,y,z)\in\mathbb{R}^2 \times \mathbb{R}_{\geq 0}$ and suppose first that $z>H^\ast(x,y)$. Then we have that:
$$f^\ast(x,y,z)=-T^\ast(x,y)(z-H^\ast(x,y))z^2 - (z-H^\ast(x,y))^2.$$
However, by assumption the second term is strictly negative and the first term is non-positive (allowing for the possibility $T^\ast(x,y)=0$). It follows that the above is strictly negative and, thus, nonzero.

Next, suppose that $z \leq H^\ast(x,y)$. Then we have
$$f^\ast(x,y,z)=-T^\ast(x,y)(z-H^\ast(x,y))z^2,$$
which equals zero if and only if either $T^\ast(x,y)=0$, $z=H^\ast(x,y)$, or $z=0$. However, unpacking the definitions we see that
$$T^\ast(x,y)=0 \quad \implies\quad (x,y)=(0,0) \quad\implies H^\ast(x,y) = 0 \quad \implies \quad 0\leq z \leq H^\ast(x,y)=0 \quad \implies \quad z=0.$$
Thus $f^\ast(x,y,z)=0$ if and only if either $z=H^\ast(x,y)$ or $z=0$, as desired.

##### Proof of Property 2

Assume $x,y\in \mathbb{R}_{\geq 0}$ and $z\in (0,1)$. Differentiating $f^\ast$ with respect to $z$ yields:

$$\frac{\partial f^\ast}{\partial z}(x,y,z)=-T^\ast(x,y)(3z-2H^\ast(x,y))z-\mathbb{1}[z> H^\ast(x,y)]2(z-H^\ast(x,y))$$

Plugging in $z=0$ and noting that $H^\ast(x,y)\geq 0$ by definition, we see that the above equals zero. Now, consider the partial derivative with respect to $x$.

$$\frac{\partial f^\ast}{\partial x}(x,y,z)=-\frac{\partial T^\ast}{\partial x}(x,y)(z-H^\ast(x,y))z^2+T^\ast(x,y)\frac{\partial H^\ast}{\partial x}(x,y)z^2+\mathbb{1}[z>H^\ast(x,y)]H^\ast(x,y)\frac{\partial H^\ast}{\partial x}(x,y)$$

Again substituting $z=0$ yields the partial derivative is zero. A similar derivation holds for the partial derivative with respect to $y$.

##### Proof of Property 3 

Assume $x,y\in \mathbb{R}_{\geq 0}$ and $z\in (0,1)$, and consider the partial derivative of $f^\ast$ with respect to $x$ given in the proof of Property 2.

Suppose it were the case that $z > H^\ast(x,y)$. Then we have that $3z-2H^\ast(x,y)\geq H^\ast(x,y) \geq 0$ and $2(z-H^\ast(x,y))>0$. As $T^\ast(x,y)\geq 0$ by construction, it follows that the above partial derivative is the sum of a non-positive term and a strictly negative term. It follows that it is strictly negative, and thus nonzero.

Now, suppose that $z \leq H^\ast(x,y)$. Then the above is equal to zero if and only if one of the following conditions hold:
 - $T^\ast(x,y)=0$
 - $3z-2H^\ast(x,y)=0$
 - $z=0$

In the first case, note that $T^\ast(x,y) = 0$ if and only if $x=y=0$. However, this implies $H^\ast(x,y)=0<z$, which contradicts the assumption that $z \leq H^\ast(x,y)$. Therefore, the first case cannot happen. Similarly, the third case is ruled out by the fact that $z\in (0,1)$. Thus, we need only consider the second.

Next, consider the partial derivative with respect to $x$. Evaluating this at $z=\frac{2}{3}H^\ast(x,y)$ gives the following:

$$\frac{\partial f^\ast}{\partial x}\bigg(x,y,\frac{2}{3}H^\ast(x,y)\bigg)=\frac{4}{27}\frac{\partial T^\ast}{\partial x}(x,y)H^\ast(x,y)^3+\frac{4}{9}(T^\ast(x,y))\frac{\partial H^\ast}{\partial x}(x,y)H^\ast(x,y)^2$$

\begin{equation}\nonumber
=\frac{4}{9}H^\ast(x,y)^2\bigg(\frac{1}{3}\frac{\partial T^\ast}{\partial x}(x,y)H^\ast(x,y)+T^\ast(x,y)\frac{\partial H^\ast}{\partial x}(x,y)\bigg).
\end{equation}

By assumption we have $z>0$, and thus $H^\ast(x,y)=\frac{3}{2}z>0$. Suppose $(x,y)\in (0,1)\times [0,1)$. Then 

\begin{equation}
\frac{\partial T^\ast}{\partial x}(x,y)=\tilde{S}'(x)>0, \quad T^\ast(x,y)=\frac{1}{2}(\tilde{S}(x)+\tilde{S}(y))>0\quad\text{ and }\quad\frac{\partial H^\ast}{\partial x}(x,y)=\tilde{H}'(x)\geq 0.
\end{equation}
Thus, we have that the partial derivative with respect to $x$ is positive. By identical reasoning, we have that if $(x,y)\in [0,1)\times (0,1)$ then:
$$\frac{\partial f^\ast}{\partial y}\bigg(x,y,\frac{2}{3}H^\ast(x,y)\bigg)>0.$$

Combining the above we see that the only way for the partial derivatives with respect to $x,y$ and $z$ to be simultaneously zero is if:

$$(x>1 \quad\text{or}\quad y> 1)\quad \text{and}\quad z=\frac{2}{3}H^\ast(x,y).$$

(Note that the case $x=y=0$ is ruled out as this would imply $z=\frac{2}{3}H^\ast(x,y)=0$, which would contradict the assumption that $z>0$.) The above conditions, when taken together imply that $z\geq \frac{1}{3}$. Finally, we note that we can extend this argument to $x,y\in \mathbb{R}$ by noting that $f^\ast$ is even in $x$ and $y$. It now follows that $\nabla f^\ast(x,y,z) \neq 0$ for $z\in(0,\frac{1}{3})$.

We are now in a position to define $f:[0,\infty) \times [0,\pi] \times [0,2\pi)\rightarrow \mathbb{R}$ in terms of spherical coordinates as:

$$
f(r,\theta,\psi) := f^*\left(\frac{\pi}{2k}(\theta + k),\, \frac{\pi}{k}(\psi + k), 3r\right)
$$

We note that, by construction, $f(0,\theta,\psi)=0$ for all $\theta$ and $\psi$. Further, for $(r,\theta,\psi)\not\in [0,\infty) \times[\frac{\pi}{4},\frac{3\pi}{4}]\times [\frac{\pi}{2},\frac{3\pi}{2}]$ we have

$$f(r,\theta,\psi)=-9(3r-1)r^2 -\mathbb{1}[3r> 1](3r-1)^2.$$

From this, it is easily follows that $f$ is a well defined function in terms of spherical coordinates. Further, by considering properties 2 and 3 of $f^\ast$, we see that $f$ is $C^1$ and has a single critical point at the origin. Finally, noting property 1, we see that $f^{-1}(0)$ is the desired level set.

The below code plots slices of $f$ along the (Cartesian) $z$-axis.

In [ ]:
# Define the z-slice to visualize
z_slice = 0

# Set resolution (higher value = finer detail, but slower rendering)
res = 300

# Plot a 2D slice of f at the specified z-value
plot_z_slice_of_f(z_slice, res)

This concludes the construction.